# Traveling Ethiopia Search Problem

This notebook addresses the Traveling Ethiopia search problem using various search algorithms. The problem is divided into four main questions, each focusing on different aspects of search strategies.

In [1]:
import heapq
import numpy as np

# Class to Manage data Structure i.e Graphs
class Graph:
    def __init__(self, connections=None):
        self.graph = connections if connections else {}
    def get_connections(self):
        return self.graph
    def add_connection(self, node, connected_nodes):
        self.graph[node] = connected_nodes
    # Modified to handle graph 1
    def get_connected_nodes(self, node):
        return self.graph.get(node, [])
     # add a method to separate edge costs from nodes
    def get_connected_nodes_with_costs(self, node):
        return self.graph.get(node, [])

# Search Method Class
class Search:
    def __init__(self, graph):
       self.graph = graph

   # Breadth First Search
    def bfs(self,start, goal):
        queue = [(start, [start])]
        visited  = set()
        while queue:
            (node, path) = queue.pop(0)
            if node == goal:
                return path
            if node not in visited:
                visited.add(node)
                for neighbor in self.graph.get_connected_nodes(node):
                    queue.append((neighbor, path + [neighbor] ))
        return None
        
    # Depth First Search
    def dfs(self, start, goal):
        stack = [(start, [start])]
        visited = set()
        while stack:
          (node, path) = stack.pop()
          if node == goal:
             return path
          if node not in visited:
                visited.add(node)
                for neighbor in self.graph.get_connected_nodes(node):
                  stack.append((neighbor, path + [neighbor]))
        return None

 # Uniform Cost search
    def uniform_cost_search(self, start, goal):
            priority_queue = [(0, start, [start])]  # (cost, node, path)
            visited = set()

            while priority_queue:
                cost, current_node, path = heapq.heappop(priority_queue)

                if current_node == goal:
                     return path

                if current_node in visited:
                    continue
                visited.add(current_node)

                for neighbor, edge_cost in self.graph.get_connected_nodes(current_node):
                    new_cost = cost + edge_cost
                    heapq.heappush(priority_queue, (new_cost, neighbor, path + [neighbor]))

            return None


       # Customized Uniform Cost Search for multiple Goals
    def custom_uniform_cost_search(self, start, goals):
        print(f"Starting custom UCS path. Initial node: {start}, goals: {goals}")
        for depth in range(1, 100):  # Set a reasonable max depth to prevent infinite loop
            print(f"Searching with depth: {depth}")
            result = self._dfs_multiple_goals(start, goals, depth, [], set())
            if result:
                print(f"All goals found. Returning Path.")
                return result
        print("No path found to all goals.")
        return None

    def _dfs_multiple_goals(self, current_node, goals, depth, path, visited_goals):
      print(f"  Visiting node: {current_node}, path: {path}, depth: {depth}, visited_goals: {visited_goals}")
      if depth == 0:
          print(f"   -- Base Case Reached, path is too deep. returning None.")
          return None
      if current_node in goals and current_node not in visited_goals:
            visited_goals.add(current_node)
            print(f"   Goal Node Reached {current_node}. Visited Goals {visited_goals}")
            if visited_goals == set(goals):
                  print (f"    -- All Goals were found.")
                  return path+[current_node]

      for neighbor_cost in self.graph.get_connected_nodes_with_costs(current_node):
           neighbor, edge_cost = neighbor_cost if isinstance(neighbor_cost, tuple) else (neighbor_cost, 0)
           print (f"   -- Exploring Neighbour {neighbor}")
           result = self._dfs_multiple_goals(neighbor, goals, depth - 1, path + [current_node], visited_goals.copy())
           if result:
                     return result
      print(f"   -- No solution found. Returning None")
      return None


    # A Star Search
    def a_star_search(self, start, goal, heuristic_values):
        priority_queue = [(heuristic_values[start], 0, start, [start])] # (f_score, cost, node, path)
        visited = set()

        while priority_queue:
              f_score, cost, current_node, path = heapq.heappop(priority_queue)

              if current_node == goal:
                  return path
              if current_node in visited:
                  continue
              visited.add(current_node)
              for neighbor, edge_cost in self.graph.get_connected_nodes(current_node):
                    new_cost = cost + edge_cost
                    new_f_score = new_cost + heuristic_values.get(neighbor, 0) # if no heuristic is provided set it to zero
                    heapq.heappush(priority_queue, (new_f_score, new_cost, neighbor, path + [neighbor]))
        return None

    # Minimax Search with Path
    def minimax(self, node, depth, maximizingPlayer, graph, path=[]):
        if depth == 0 or node not in graph.get_connections():
            return self.get_utility(node, graph), path + [node]

        if maximizingPlayer:
            maxEval = float('-inf')
            best_path = []
            for child in graph.get_connected_nodes(node):
                eval, current_path = self.minimax(child, depth - 1, False, graph,path + [node]) # Pass the existing path
                if isinstance(eval,int) and isinstance(maxEval,float) and eval > maxEval:
                     maxEval = eval
                     best_path = current_path
                elif isinstance(eval,float) and isinstance(maxEval,str) and eval > float(maxEval):
                     maxEval = eval
                     best_path = current_path
                elif isinstance(eval,float) and isinstance(maxEval,float) and eval > maxEval:
                     maxEval = eval
                     best_path = current_path



            return maxEval,best_path
        else:
            minEval = float('inf')
            best_path= []
            for child in graph.get_connected_nodes(node):
                eval , current_path = self.minimax(child, depth - 1, True, graph,path + [node]) # Pass the existing path
                if isinstance(eval,int) and isinstance(minEval,float) and eval < minEval:
                      minEval = eval
                      best_path = current_path
                elif isinstance(eval,float) and isinstance(minEval,str) and eval < float(minEval):
                      minEval = eval
                      best_path = current_path
                elif isinstance(eval,float) and isinstance(minEval,float) and eval < minEval:
                     minEval = eval
                     best_path = current_path
            return minEval, best_path
    def get_utility(self, node, graph):
     for key, val in graph.get_connections().items():
        if key == node:
            if isinstance(val, list) and val:
                try:
                    # Attempt to convert val[0] to an integer
                    return int(val[0])
                except ValueError:
                    # Handle the case where val[0] is not a valid integer
                    # You might return a default value or handle it differently
                    return 0  # or some other default value

# 2.Data Structures
## 2.1 Conversion of Figure 1 to a Dictionary
figure1_connections = {
    'Kartum': ['Tumer'],
    'Tumer': ['Shire', 'Kartum'],
    'Shire': ['Axum', 'Tumer', 'Debark'],
    'Axum': ['Shire','Adigrat'],
    'Adigrat': ['Axum', 'Mekele'],
    'Mekele': ['Adigrat', 'Alamata', "Debre Tabor"],
    'Alamata': ['Mekele', 'Lalibela', 'Dessie'],
    'Debark': ['Shire', 'Gondar', 'Metema'],
    'Gondar': ['Debark', 'Bahir Dar','Azazo'],
    'Azazo':['Gondar', 'Metekel'],
    'Metekel':['Azazo','Bahir Dar'],
    'Bahir Dar': ['Gondar', 'Metekel','Debre Markos'],
    'Debre Tabor': ['Mekele', 'Lalibela', 'Dessie'],
    'Lalibela':['Debre Tabor', 'Alamata', 'Dessie'],
    'Dessie': ['Lalibela', 'Debre Sina', 'Alamata', 'Kombolcha'],
    'Debre Markos':['Bahir Dar', 'Finote Selam', 'Debre Birhan'],
    'Finote Selam': ['Debre Markos', "Ambo"],
    'Ambo': ['Finote Selam', 'Addis Ababa', 'Nekemte'],
    'Addis Ababa': ['Ambo', 'Debre Sina', 'Bishoftu', 'Adama'],
     'Debre Sina':['Dessie', 'Addis Ababa','Kombolcha'],
    'Kombolcha': ['Dessie', 'Debre Sina',  'Dire Dawa'],
    'Nekemte':['Ambo', 'Gimbi','Bako'],
    'Gimbi': ['Nekemte', 'Gambella', 'Dembidollo'],
    'Gambella':[ 'Gimbi', 'Gore', 'Bonga'],
    'Gore':['Gambella', 'Tepi'],
    'Tepi':['Gore','Bonga'],
    'Bonga':['Gambella', 'Tepi', 'Jimma', 'Dawro'],
    'Jimma': ['Bonga', 'Welkite', 'Hossana'],
    'Dembidollo': ['Gimbi', 'Nekemte'],
    'Welkite':['Jimma', 'Hossana', 'Butajira'],
    'Hossana': ['Welkite','Jimma','Shashemene'],
    'Butajira' : ['Welkite', 'Shashemene', 'Batu' ],
    'Shashemene': ['Hossana', 'Butajira', 'Assassa', 'Hawassa'],
    'Hawassa':['Shashemene','Dilla'],
    'Adama': ['Addis Ababa', 'Bishoftu', 'Mojo', 'Awash'],
    'Bishoftu': ['Adama', 'Addis Ababa', 'Batu'],
    'Batu': ['Bishoftu', 'Butajira','Assassa', 'Awash'],
    'Assassa':['Batu','Shashemene','Dodola','Goba'],
     'Awash':['Adama', 'Batu', 'Dire Dawa'],
    'Dire Dawa': ['Kombolcha','Awash', 'Harar'],
    'Harar' : ['Dire Dawa','Babile'],
    'Babile' : ['Harar', 'Jigjiga'],
    'Jigjiga' : [ 'Babile', 'Degehabur' ],
     'Dodola': [ 'Assassa', 'Bale'],
    'Bale' : ['Dodola', 'Goba', 'Liben'],
    'Goba' : ['Assassa', 'Bale','Sof Oumer','Dega Habur'],
     'Sof Oumer': ['Goba','Liben','Gode'],
     'Dilla':['Hawassa','Yabello'],
     'Liben':['Bale', 'Sof Oumer', 'Moyale'],
     'Gode':['Sof Oumer', 'Mokadisho', 'Warder'],
      'Yabello':['Dilla','Konso'],
      'Konso': ['Yabello', 'Moyale'],
      'Moyale':['konso', 'Liben', 'Nairobi'],
      'Dega Habur':['Jigjiga','Gode'],
      'Warder':['Dega Habur','Gode'],
    'Mokadisho':['Gode']
}
## 2.2 Conversion of Figure 2 to a a Dictionary
figure2_connections = {
     'Kartum': [('Tumer', 18), ('Gonder',21)],
     'Tumer': [('Shire', 7), ('Kartum', 18)],
    'Shire': [('Axum', 8), ('Tumer',7), ('Debark',9)],
    'Axum': [('Shire', 8),('Adigrat',6)],
    'Adigrat': [('Axum', 6), ('Mekele',6)],
    'Mekele': [('Adigrat', 6), ('Alamata', 6), ("Debre Tabor", 7)],
    'Alamata': [('Mekele',6), ('Lalibela', 7), ('Dessie', 11)],
    'Debark': [('Shire', 9), ('Gondar', 7), ('Metema',8)],
    'Gondar': [('Debark', 7), ('Bahir Dar', 4),('Azazo', 7)],
    'Azazo':[('Gondar', 7), ('Metekel', 7)],
    'Metekel':[('Azazo', 7),('Bahir Dar', 11)],
    'Bahir Dar': [('Gondar', 4), ('Metekel', 11),('Debre Markos', 8)],
    'Debre Tabor': [('Mekele', 7), ('Lalibela', 8), ('Dessie', 11)],
    'Lalibela':[('Debre Tabor', 8), ('Alamata', 7), ('Dessie', 4)],
    'Dessie': [('Lalibela', 4), ('Debre Sina', 4), ('Alamata', 11), ('Kombolcha', 7)],
    'Debre Markos':[('Bahir Dar', 8), ('Finote Selam', 3), ('Debre Birhan', 17)],
    'Finote Selam': [('Debre Markos', 3), ("Ambo", 5)],
    'Ambo': [('Finote Selam', 5), ('Addis Ababa', 9), ('Nekemte', 11)],
    'Addis Ababa': [('Ambo', 9), ('Debre Sina', 2), ('Bishoftu', 3), ('Adama', 5)],
     'Debre Sina':[('Dessie', 4), ('Addis Ababa', 2),('Kombolcha', 6)],
    'Kombolcha': [('Dessie', 7), ('Debre Sina', 6),  ('Dire Dawa', 5)],
    'Nekemte':[('Ambo', 11), ('Gimbi', 4),('Bako', 7)],
    'Gimbi': [('Nekemte', 4), ('Gambella', 5), ('Dembidollo', 6)],
    'Gambella':[('Gimbi', 5), ('Gore', 9), ('Bonga', 8)],
    'Gore':[('Gambella', 9), ('Tepi', 8)],
    'Tepi':[('Gore', 8),('Bonga', 6)],
    'Bonga':[('Gambella', 8), ('Tepi', 6), ('Jimma', 4), ('Dawro', 10)],
    'Jimma': [('Bonga', 4), ('Welkite', 7), ('Hossana', 4)],
    'Dembidollo': [('Gimbi', 6), ('Nekemte', 8)],
    'Welkite':[('Jimma', 7), ('Hossana', 7), ('Butajira', 3)],
    'Hossana': [('Welkite', 7),('Jimma', 4),('Shashemene', 4)],
    'Butajira' : [('Welkite', 3), ('Shashemene', 4), ('Batu', 5)  ],
    'Shashemene': [('Hossana', 4), ('Butajira', 4), ('Assassa', 5), ('Hawassa', 3)],
    'Hawassa':[('Shashemene', 3),('Dilla', 6)],
    'Adama': [('Addis Ababa', 5), ('Bishoftu', 3), ('Mojo', 2), ('Awash', 5)],
    'Bishoftu': [('Adama', 3), ('Addis Ababa', 3), ('Batu', 2)],
    'Batu': [('Bishoftu', 2), ('Butajira', 5),('Assassa', 4), ('Awash', 3)],
    'Assassa':[('Batu', 4),('Shashemene', 5),('Dodola', 6),('Goba',18)],
     'Awash':[('Adama', 5), ('Batu', 3), ('Dire Dawa', 4)],
    'Dire Dawa': [('Kombolcha', 5),('Awash', 4), ('Harar', 4)],
    'Harar' : [('Dire Dawa',4),('Babile', 2)],
    'Babile' : [('Harar', 2), ('Jigjiga', 3)],
    'Jigjiga' : [('Babile', 3), ('Dega Habur', 5)],
     'Dodola': [('Assassa', 6), ('Bale', 13 )],
    'Bale' : [('Dodola', 13), ('Goba', 18), ('Liben', 11)],
    'Goba' : [('Assassa', 18), ('Bale', 18),('Sof Oumer', 23),('Dega Habur', 5)],
     'Sof Oumer': [('Goba', 23),('Liben', 23),('Gode', 23)],
     'Dilla':[('Hawassa', 6),('Yabello',3)],
     'Liben':[('Bale', 11), ('Sof Oumer', 23),('Moyale',11)],
     'Gode':[('Sof Oumer', 23),('Mokadisho', 17),('Warder', 6)],
      'Yabello':[('Dilla', 3),('Konso', 4)],
      'Konso': [('Yabello', 4), ('Moyale', 6)],
      'Moyale':[('konso', 6), ('Liben', 11),('Nairobi',22)],
      'Dega Habur':[('Jigjiga', 5),('Gode', 6)],
      'Warder':[('Dega Habur', 6),('Gode', 6)],
    'Mokadisho':[('Gode',17)]
}


## 2.3 Conversion of Figure 3 to a Dictionary
figure3_connections = {
    'Kartum': [('Tumer', 21)],
     'Tumer': [('Shire', 7), ('Kartum', 21)],
    'Shire': [('Axum', 8), ('Tumer',7), ('Debark',9)],
    'Axum': [('Shire', 8),('Adigrat',6)],
    'Adigrat': [('Axum', 6), ('Mekele',6)],
    'Mekele': [('Adigrat', 6), ('Alamata', 6), ("Debre Tabor", 7)],
    'Alamata': [('Mekele',6), ('Lalibela', 7), ('Dessie', 11)],
    'Debark': [('Shire', 9), ('Gondar', 7), ('Metema',8)],
    'Gondar': [('Debark', 7), ('Bahir Dar', 4),('Azazo', 7)],
    'Azazo':[('Gondar', 7), ('Metekel', 7)],
    'Metekel':[('Azazo', 7),('Bahir Dar', 11)],
    'Bahir Dar': [('Gondar', 4), ('Metekel', 11),('Debre Markos', 8)],
    'Debre Tabor': [('Mekele', 7), ('Lalibela', 8), ('Dessie', 11)],
    'Lalibela':[('Debre Tabor', 8), ('Alamata', 7), ('Dessie', 4)],
    'Dessie': [('Lalibela', 4), ('Debre Sina', 4), ('Alamata', 11), ('Kombolcha', 7)],
    'Debre Markos':[('Bahir Dar', 8), ('Finote Selam', 3), ('Debre Birhan', 27)],
    'Finote Selam': [('Debre Markos', 3), ("Ambo", 5)],
    'Ambo': [('Finote Selam', 5), ('Addis Ababa', 9), ('Nekemte', 11)],
    'Addis Ababa': [('Ambo', 9), ('Debre Sina', 2), ('Bishoftu', 3), ('Adama', 5)],
     'Debre Sina':[('Dessie', 4), ('Addis Ababa', 2),('Kombolcha', 6)],
    'Kombolcha': [('Dessie', 7), ('Debre Sina', 6),  ('Dire Dawa', 5)],
    'Nekemte':[('Ambo', 11), ('Gimbi', 4),('Bako', 7)],
    'Gimbi': [('Nekemte', 4), ('Gambella', 5), ('Dembidollo', 6)],
    'Gambella':[('Gimbi', 5), ('Gore', 9), ('Bonga', 8)],
    'Gore':[('Gambella', 9), ('Tepi', 8)],
    'Tepi':[('Gore', 8),('Bonga', 6)],
    'Bonga':[('Gambella', 8), ('Tepi', 6), ('Jimma', 4), ('Dawro', 10)],
    'Jimma': [('Bonga', 4), ('Welkite', 7), ('Hossana', 4)],
    'Dembidollo': [('Gimbi', 6), ('Nekemte', 8)],
    'Welkite':[('Jimma', 7), ('Hossana', 7), ('Butajira', 3)],
    'Hossana': [('Welkite', 7),('Jimma', 4),('Shashemene', 4)],
    'Butajira' : [('Welkite', 3), ('Shashemene', 4), ('Batu', 5)  ],
    'Shashemene': [('Hossana', 4), ('Butajira', 4), ('Assassa', 5), ('Hawassa', 3)],
    'Hawassa':[('Shashemene', 3),('Dilla', 6)],
    'Adama': [('Addis Ababa', 5), ('Bishoftu', 3), ('Mojo', 2), ('Awash', 5)],
    'Bishoftu': [('Adama', 3), ('Addis Ababa', 3), ('Batu', 2)],
    'Batu': [('Bishoftu', 2), ('Butajira', 5),('Assassa', 4), ('Awash', 3)],
    'Assassa':[('Batu', 4),('Shashemene', 5),('Dodola', 6),('Goba',18)],
     'Awash':[('Adama', 5), ('Batu', 3), ('Dire Dawa', 4)],
    'Dire Dawa': [('Kombolcha', 5),('Awash', 4), ('Harar', 4)],
    'Harar' : [('Dire Dawa',4),('Babile', 2)],
    'Babile' : [('Harar', 2), ('Jigjiga', 3)],
    'Jigjiga' : [('Babile', 3), ('Dega Habur', 5)],
     'Dodola': [('Assassa', 6), ('Bale', 13 )],
    'Bale' : [('Dodola', 13), ('Goba', 18), ('Liben', 11)],
    'Goba' : [('Assassa', 18), ('Bale', 18),('Sof Oumer', 23),('Dega Habur', 5)],
     'Sof Oumer': [('Goba', 23),('Liben', 23),('Gode', 23)],
     'Dilla':[('Hawassa', 6),('Yabello',3)],
     'Liben':[('Bale', 11), ('Sof Oumer', 23),('Moyale',11)],
     'Gode':[('Sof Oumer', 23),('Mokadisho', 17),('Warder', 6)],
      'Yabello':[('Dilla', 3),('Konso', 4)],
      'Konso': [('Yabello', 4), ('Moyale', 6)],
        'Moyale':[('konso', 6), ('Liben', 11)],
      'Dega Habur':[('Jigjiga', 5),('Gode', 6)],
      'Warder':[('Dega Habur', 6),('Gode', 6)],
    'Mokadisho':[('Gode',17)]
}
figure3_heuristic_values = {
    'Kartum': 68,
    'Tumer': 67,
    'Shire': 66,
    'Axum': 63,
    'Adigrat': 61,
    'Mekele': 58,
    'Alamata': 52,
    'Debark': 56,
    'Gondar': 54,
    'Azazo':51,
    'Metekel':50,
     'Bahir Dar': 50,
    'Debre Tabor': 50,
   'Lalibela': 45,
    'Dessie': 43,
    'Debre Markos':49,
    'Finote Selam': 47,
    'Ambo': 45,
    'Addis Ababa': 41,
    'Debre Sina': 38,
    'Kombolcha': 38,
    'Nekemte': 39,
    'Gimbi': 40,
    'Gambella': 38,
    'Gore': 35,
    'Tepi': 33,
    'Bonga': 32,
    'Jimma': 36,
    'Dembidollo': 40,
    'Welkite': 30,
    'Hossana': 28,
    'Butajira':25,
    'Shashemene': 23,
    'Hawassa': 22,
    'Adama': 34,
    'Bishoftu': 30,
    'Batu': 26,
    'Assassa': 24,
    'Awash': 31,
     'Dire Dawa': 24,
    'Harar': 20,
     'Babile': 17,
    'Jigjiga': 15,
     'Dodola': 21,
    'Bale': 18,
     'Goba': 17,
    'Sof Oumer': 16,
    'Dilla':19,
    'Liben': 11,
    'Gode': 10,
    'Yabello': 10,
    'Konso': 6,
    'Moyale': 0,
    'Dega Habur': 9,
    'Warder': 6,
    'Mokadisho': 20
}
## 2.4 Conversion of Figure 4 to a Dictionary
figure4_connections = {
    'Addis Ababa': ['Ambo', 'Adama'],
    'Ambo': ['Gedo', 'Nekemte', 'Butajira'],
    'Adama': ['Mojo', 'DireDawa'],
    'Gedo': ['Finca','Shambu'],
    'Nekemte':['Gimbi','Limu'],
    'Butajira':['Worabe','Wolkite'],
    'Mojo':['Dilla','Kaffa'],
    'DireDawa':['Harar','Chiro'],
     'Worabe':['Hossana','Tepi'],
    'Wolkite':['Bench Maji', "Tepi"],
    'Harar': [10],
    'Chiro':[6],
    'Shambu':[4],
    'Finca':[5],
    'Gimbi':[8],
    'Limu':[6],
     'Hossana':[6],
     'Tepi':[6],
    'Bench Maji':[5],
    'Dilla':[9],
    'Kaffa':[7]
}

# initialize the graph objects
graph1 = Graph(figure1_connections)
graph2 = Graph(figure2_connections)
graph3 = Graph(figure3_connections)
graph4 = Graph(figure4_connections)
# Initialize the search class
Search1 = Search(graph1)
Search2 = Search(graph2)
Search3 = Search(graph3)
Search4 = Search(graph4)

 # Solve the problems
# Question 1:
print("Question 1:")
print("  1.1: Successfully converted Figure 1 to a Dictionary.")
initial_state_q1 = 'Kartum'
goal_state_q1 = 'Moyale'
print("   1.2: BFS Path:", Search1.bfs(initial_state_q1, goal_state_q1))
print("   1.2: DFS Path:",Search1.dfs(initial_state_q1, goal_state_q1))

# Question 2:
print("\nQuestion 2:")
print("  2.1: Successfully converted Figure 2 to a Dictionary.")
initial_state_q2_ucs = "Addis Ababa"
goal_state_q2_ucs = "Lalibela"
print("  2.2: UCS Path from " + initial_state_q2_ucs +" to " + goal_state_q2_ucs+ " :", Search2.uniform_cost_search(initial_state_q2_ucs, goal_state_q2_ucs))
goal_states_q2_custom = ["Axum", "Gondar", "Lalibela", "Babile", "Jimma", "Bale", "Sof Oumer", "Arba Minch"]
print("  2.3: Customized UCS Path to Multiple Goals: ",Search2.custom_uniform_cost_search(initial_state_q2_ucs, goal_states_q2_custom ))

#Question 3:
print ("\n Question 3:")
print(" Successfully converted Figure 3 to a Dictionary")
initial_state_q3 = 'Addis Ababa'
goal_state_q3 = 'Moyale'
print("   A* Path From "+initial_state_q3+" to "+goal_state_q3+":", Search3.a_star_search(initial_state_q3, goal_state_q3,figure3_heuristic_values))

#Question 4:
print ("\n Question 4:")
print("  Successfully converted Figure 4 to a Dictionary")
initial_state_q4 = "Addis Ababa"
depth_q4 = 3
best_value, best_path  =  Search4.minimax(initial_state_q4,depth_q4,True, graph4)
print(f"   Best achievable utility value for the coffee: {best_value}")
print(f"    Best Path : {best_path}")


Question 1:
  1.1: Successfully converted Figure 1 to a Dictionary.
   1.2: BFS Path: ['Kartum', 'Tumer', 'Shire', 'Axum', 'Adigrat', 'Mekele', 'Alamata', 'Dessie', 'Debre Sina', 'Addis Ababa', 'Bishoftu', 'Batu', 'Assassa', 'Dodola', 'Bale', 'Liben', 'Moyale']
   1.2: DFS Path: ['Kartum', 'Tumer', 'Shire', 'Debark', 'Gondar', 'Azazo', 'Metekel', 'Bahir Dar', 'Debre Markos', 'Finote Selam', 'Ambo', 'Nekemte', 'Gimbi', 'Gambella', 'Bonga', 'Jimma', 'Hossana', 'Shashemene', 'Hawassa', 'Dilla', 'Yabello', 'Konso', 'Moyale']

Question 2:
  2.1: Successfully converted Figure 2 to a Dictionary.
  2.2: UCS Path from Addis Ababa to Lalibela : ['Addis Ababa', 'Debre Sina', 'Dessie', 'Lalibela']

 Question 3:
 Successfully converted Figure 3 to a Dictionary
   A* Path From Addis Ababa to Moyale: ['Addis Ababa', 'Bishoftu', 'Batu', 'Assassa', 'Shashemene', 'Hawassa', 'Dilla', 'Yabello', 'Konso', 'Moyale']

 Question 4:
  Successfully converted Figure 4 to a Dictionary
   Best achievable utility v